In [75]:
from __future__ import unicode_literals, print_function, division
from io import open
import numpy as np
import unicodedata
import string
import re
import random
import torchtext
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
from google.colab import drive
import nltk as nl
from tqdm import tqdm  
from matplotlib import pyplot as plt
import datetime
import time
import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import gensim.downloader as api
! pip install pytorch-pretrained-bert
! pip install transformers


In [76]:

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# load BERT tokenizer
tz = BertTokenizer.from_pretrained("bert-base-cased")

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [77]:

p_intent = []
p_snippet = []
c = 0
iit = []
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent:
        iit.append(c)
      c+=1
      p_intent.append(i[:-1].lower())
p_intentn = []
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentn.append(i[:-1].lower())
with open('y.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippet.append(i.lower())
  
train_pairs = [[p_intentn[i],p_snippet[i]] for i in iit]
p_intentval = []
p_snippetval = []
iiv = []
c= 0
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intentval:
        iiv.append(c)
      
      p_intentval.append(i[:-1].lower())
      c+=1
p_intentvaln = []
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentvaln.append(i[:-1].lower())
      c+=1
with open('yv.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      #i = i.split(' ')
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      p_snippetval.append(i.lower())
val_pairs = [[p_intentvaln[i],p_snippetval[i]] for i in iiv]
p_intenttest = []
p_snippettest = []
ii = []
c= 0
with open('xt.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intenttest:
        ii.append(c)
      c+=1
      p_intenttest.append(i[:-1].lower())
with open('yt.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      p_snippettest.append(i.lower())
test_pairs = [[p_intenttest[i],p_snippettest[i]] for i in ii]
p_intenttest1 = []
p_snippettest1 = []
with open('xt1.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest1.append(i[:-1].lower())
with open('yt1.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest1.append(i.lower())
test_pairs1 = [[p_intenttest1[i],p_snippettest1[i]] for i in ii]
p_intenttest2 = []
p_snippettest2 = []
with open('xt2.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest2.append(i[:-1].lower())
with open('yt2.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest2.append(i.lower())
test_pairs2 = [[p_intenttest2[i],p_snippettest2[i]] for i in ii]

In [78]:
# load extra training data
# p_intentextra = []
# p_snippetextra = []
# c = 0
# iite = []
# with open('xextra.txt', 'r',encoding='UTF-8') as intent:
#     for i in intent:
#       if i[:-1].lower() not in p_intentextra and i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intenttest and i[:-1].lower() not in p_intentval:
#         iite.append(c)
#       c+=1
#       p_intentextra.append(i[:-1].lower())

# with open('yextra.txt', 'r',encoding='UTF-8') as snippet:
#     for i in snippet:
#       p_snippetextra.append(i[:-1].lower())
# extratrain = [[p_intentextra[i],p_snippetextra[i]] for i in iite]
# train_pairs = train_pairs+extratrain

In [79]:
# use BERTtokenizer tokenize the input
k = [['[CLS]'] + tz.tokenize(x[0]) + ['[SEP]'] for x in train_pairs]
kk = [['[CLS]'] +tz.tokenize(x[0]) + ['[SEP]'] for x in val_pairs]
kkk = [['[CLS]'] + tz.tokenize(x[0]) + ['[SEP]'] for x in test_pairs]
kkk1 = [['[CLS]'] + tz.tokenize(x[0]) + ['[SEP]'] for x in test_pairs1]
kkk2 = [['[CLS]'] + tz.tokenize(x[0]) + ['[SEP]'] for x in test_pairs2]

In [80]:
# convert to index according to BERT vocab
enco = [torch.tensor(tz.convert_tokens_to_ids(x)) for x in k]
encoval =[torch.tensor(tz.convert_tokens_to_ids(x)) for x in kk]
encotest = [torch.tensor(tz.convert_tokens_to_ids(x)) for x in kkk]
encotest1 = [torch.tensor(tz.convert_tokens_to_ids(x)) for x in kkk1]
encotest2 = [torch.tensor(tz.convert_tokens_to_ids(x)) for x in kkk2]

In [81]:
tokenizer_for_tar = lambda x: x.split() # 分词器
MAX_LENGTH = 196
SRC_TEXT = torchtext.legacy.data.Field(sequential=True,

                                )
TARG_TEXT = torchtext.legacy.data.Field(sequential=True,
              
                                 )




In [82]:
# use normal tokenizer tokenize the label with white space seperator
t  = [['[CLS]'] + x[1].split() + ['[SEP]'] for x in train_pairs]
tt = [['[CLS]'] + x[1].split() + ['[SEP]'] for x in val_pairs]
ttt = [['[CLS]'] + x[1].split() + ['[SEP]'] for x in test_pairs]

In [83]:
# build vocab for encoder
SRC_TEXT.build_vocab(k)  
print(len(SRC_TEXT.vocab))
print(SRC_TEXT.vocab.itos[0])
print(SRC_TEXT.vocab.itos[1])
print(SRC_TEXT.vocab.itos[2])
print(SRC_TEXT.vocab.itos[3])
print(SRC_TEXT.vocab.stoi['[CLS]'])
print(SRC_TEXT.vocab.stoi['[SEP]'])
# build vocab for decoder
TARG_TEXT.build_vocab(t)
print(len(TARG_TEXT.vocab))
print(TARG_TEXT.vocab.itos[0])
print(TARG_TEXT.vocab.itos[1])
print(TARG_TEXT.vocab.itos[2])
print(TARG_TEXT.vocab.itos[3])
print(TARG_TEXT.vocab.stoi['[CLS]'])
print(TARG_TEXT.vocab.stoi['[SEP]'])

11040
<unk>
<pad>
[CLS]
[SEP]
2
3
19063
<unk>
<pad>
(
)
5
6


In [84]:

# convert to index according to encoder vocab
train_i = [[SRC_TEXT.vocab.stoi[x] for x in i] for i in k]
val_i = [[SRC_TEXT.vocab.stoi[x] for x in i] for i in kk]
test_i = [[SRC_TEXT.vocab.stoi[x] for x in i] for i in kkk]
test_i1 = [[SRC_TEXT.vocab.stoi[x] for x in i] for i in kkk1]
test_i2 = [[SRC_TEXT.vocab.stoi[x] for x in i] for i in kkk2]
final_trains_i = [torch.tensor(i) for i in train_i]
final_val_i = [torch.tensor(i) for i in val_i]
final_test_i = [torch.tensor(i) for i in test_i]
final_test_i1 = [torch.tensor(i) for i in test_i1]
final_test_i2 = [torch.tensor(i) for i in test_i2]

# convert to index according to decoder vocab
train_s = [[TARG_TEXT.vocab.stoi[x] for x in i] for i in t]
val_s = [[TARG_TEXT.vocab.stoi[x] for x in i] for i in tt]
test_s = [[TARG_TEXT.vocab.stoi[x] for x in i] for i in ttt]
final_trains = [torch.tensor(i) for i in train_s]
final_val_s = [torch.tensor(i) for i in val_s]
final_test_s = [torch.tensor(i) for i in test_s]


In [85]:
print(len(final_trains))
print(len(final_trains) == len(enco))
print(len(final_test_s))

51942
True
16095


In [86]:
# for collecting data and padding data during mini-batch learning
def collate_fn(data):
    bertin, inputs,targ = zip(*data)
    bertin = torch.nn.utils.rnn.pad_sequence(bertin, batch_first=True, padding_value=0)
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=1)
    targ = torch.nn.utils.rnn.pad_sequence(targ, batch_first=True, padding_value=1)
    return bertin,inputs,targ
datat = list(zip(enco,final_trains_i,final_trains))
d = torch.utils.data.DataLoader(datat, batch_size = 4,shuffle = False,collate_fn= collate_fn)

In [87]:
for step, (b,i,t) in enumerate(d):
  with torch.no_grad():
      print(b)
      print(i)
      # enc_output = bret(i.to(device))[0]
  break
step

tensor([[  101,  1175,  1674,  1136,  4056,   170,  2458,   102],
        [  101,  1175,  1674,  1136,  4056,   170,  2616,   102],
        [  101,  1175,  1674,  1136,  4056,   170,  1411,   102],
        [  101,  1175,  1674,  1136,  4056,   170, 12629,   102]])
tensor([[   2,   14,   57,    9,   50,    6,  460,    3],
        [   2,   14,   57,    9,   50,    6,  290,    3],
        [   2,   14,   57,    9,   50,    6,  315,    3],
        [   2,   14,   57,    9,   50,    6, 5600,    3]])


0

In [88]:
def get_angles(pos, i, d_model):
    # 2*(i//2) make sure that calculating 1/10000^(2i/d)
    angle_rates = 1 / np.power(10000, 2 * (i // 2) / np.float32(d_model))
    return pos * angle_rates  # [50,1]*[1,512]=>[50, 512]

def positional_encoding(position, d_model):  # d_model = embedding_dim
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],  
                            np.arange(d_model)[np.newaxis, :], 
                            d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # 2i
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # 2i+2
    pos_encoding = angle_rads[np.newaxis, ...]  
    return torch.tensor(pos_encoding, dtype=torch.float32)

In [89]:
def create_padding_mask(seq,pad):  # seq [b, seq_len]
    seq = torch.eq(seq, torch.tensor(pad)).float() 
    return seq[:, np.newaxis, np.newaxis, :]# =>[b, 1, 1, seq_len]

def create_look_ahead_mask(size):  # seq_len
    mask = torch.triu(torch.ones((size, size)), diagonal=1)
    return mask  # [seq_len, seq_len]
def create_mask(bret_inp,inp, targ):
    # encoder padding mask
    # Used in the self-attention in the encoder.
    enc_padding_mask = create_padding_mask(inp,1)  # =>[b,1,1,inp_seq_len] 
    
    # used in decoder's encoder-decoder attention
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp,1)  # =>[b,1,1,inp_seq_len] 
    
    # Used in the self-attention in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(targ.shape[-1])  # =>[targ_seq_len,targ_seq_len]
    dec_targ_padding_mask = create_padding_mask(targ,1)  # =>[b,1,1,targ_seq_len]
    combined_mask = torch.max(look_ahead_mask, dec_targ_padding_mask)  # combine 2 masks =>[b,1,targ_seq_len,targ_seq_len]
    return enc_padding_mask, combined_mask, dec_padding_mask,create_padding_mask(bret_inp,0) #The last one is BERT padding mask, and in BERT vocab, index 0 is mean the padding

In [90]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """
    
    matmul_qk = torch.matmul(q, k.transpose(-1, -2))  #[..., seq_len_q, seq_len_k]
    # scale matmul_qk
    dk = torch.tensor(k.shape[-1], dtype=torch.float32)  # depth of k
    scaled_attention_logits = matmul_qk / torch.sqrt(dk)  # [..., seq_len_q, seq_len_k]

    # add the mask to the scaled tensor.
    if mask is not None:  # mask: [batch_size, 1, 1, seq_len]
        # The mask is multiplied with -1e9 (close to negative infinity). 
        # This is done because the mask is summed with the scaled matrix 
        # multiplication of Q and K and is applied 
        # immediately before a softmax.
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = torch.nn.functional.softmax(scaled_attention_logits, dim=-1)  

    output = torch.matmul(attention_weights, v)  
    return output, attention_weights  

In [91]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0  #need to make sure each head have same dimension

        self.depth = d_model // self.num_heads 

        self.wq = torch.nn.Linear(d_model, d_model)
        self.wk = torch.nn.Linear(d_model, d_model)
        self.wv = torch.nn.Linear(d_model, d_model)

        self.final_linear = torch.nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):  # x [b, seq_len, d_model]
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = x.view(batch_size, -1, self.num_heads,
                   self.depth)  # [b, seq_len, d_model]=>[b, seq_len, num_head, depth]
        return x.transpose(1, 2)  # [b, seq_len, num_head, depth]=>[b, num_head, seq_len, depth]

    def forward(self, q, k, v, mask):  # q=k=v=x [b, seq_len, embedding_dim] embedding_dim = d_model
        batch_size = q.shape[0]

        q = self.wq(q)  # =>[batch_size, seq_len, d_model]
        k = self.wk(k)  # =>[batch_size, seq_len, d_model]
        v = self.wq(v)  # =>[batch_sizeb, seq_len, d_model]

        q = self.split_heads(q, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        k = self.split_heads(k, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        v = self.split_heads(v, batch_size)  # =>[batch_size, num_head, seq_len, depth]
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # => [batch_size, num_head, seq_len_q, depth], [batch_size, num_head, seq_len_q, seq_len_k]

        scaled_attention = scaled_attention.transpose(1, 2)  # =>[batch_size, seq_len_q, num_head, depth]
        concatenate_attention = scaled_attention.reshape(batch_size, -1, self.d_model)  # =>[batch_size, seq_len_q, d_model]

        output = self.final_linear(concatenate_attention)  # =>[batch_size, seq_len_q, d_model]
        return output, attention_weights  # [batch_size, seq_len_q, d_model], [batch_size, num_head, seq_len_q, seq_len_k]


In [92]:
def point_wise_feed_forward_network(d_model, dff):
    feed_forward_net = torch.nn.Sequential(
        torch.nn.Linear(d_model, dff),  # [b, seq_len, d_model]=>[b, seq_len, dff=2048]
        torch.nn.ReLU(),
        torch.nn.Linear(dff, d_model),  # [b, seq_len, dff=2048]=>[b, seq_len, d_model=512]
    )
    return feed_forward_net



In [93]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)  # MultiHeadAttention（padding mask）(self-attention)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm2 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)

        self.dropout1 = torch.nn.Dropout(rate)
        self.dropout2 = torch.nn.Dropout(rate)

    # x [batch_size, inp_seq_len, embedding_dim] embedding_dim =d_model
    # mask [batch_size,1,1,inp_seq_len]
    def forward(self, x, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # =>[batch_size, seq_len, d_model]
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)  # =>[batch_size, seq_len, d_model]

        ffn_output = self.ffn(out1)  # =>[batch_size, seq_len, d_model]
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  #  =>[batch_size, seq_len, d_model]

        return out2  # [batch_size, seq_len, d_model]

In [94]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model,num_heads)  # (self-attention)
        self.mha2 = MultiHeadAttention(d_model, num_heads)  # (encoder-decoder attention)
        self.mha3 = MultiHeadAttention(d_model, num_heads)# (BERT-decoder attention)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm2 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.layernorm3 = torch.nn.LayerNorm(normalized_shape=d_model, eps=1e-6)
        self.mapp = nn.Linear(768,d_model)# Mapping funtion
        self.rrr = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(rate)
        self.dropout2 = torch.nn.Dropout(rate)
        self.dropout3 = torch.nn.Dropout(rate)
        self.dropout4 = torch.nn.Dropout(rate)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask,bert_out,b_m):
        # bert_out =  self.mapp(bert_out)
        attn1, attn_weights_block1 = self.mha1(x, x, x,
                                               look_ahead_mask) 
        attn1 = self.dropout1(attn1)
        out1 = self.layernorm1(x + attn1) 

        attn2, attn_weights_block2 = self.mha2(out1, enc_output, enc_output,
                                               padding_mask) 
        attn2 = self.dropout2(attn2)
        # bret_atten ,attn_weights_block3 =  self.mha3(out1, self.rrr(self.mapp(bert_out)), self.rrr(self.mapp(bert_out)), b_m)  # calculate BERT-decoder attention, if d_model is not match the dim of BERT, use a ReLU and linear function map the dim of BERT to d_model
        bret_atten ,attn_weights_block3 =  self.mha3(out1, bert_out, bert_out, b_m)  # calculate BERT-decoder attention
        bret_atten = self.dropout3(bret_atten)
        # average encoder-decoder attention and BERT-decoder attention
        attn3 = (attn2+bret_atten)/2
        out2 = self.layernorm2(out1 + attn3) 
        ffn_output = self.ffn(out2)  # =>[b, targ_seq_len, d_model]
        ffn_output = self.dropout4(ffn_output)
        out3 = self.layernorm3(out2 + ffn_output) 

        return out3, attn_weights_block1, (attn_weights_block2+attn_weights_block3)/2

In [95]:
class Encoder(torch.nn.Module):
    def __init__(self,
                 num_layers,  # N个encoder layer
                 d_model,
                 num_heads,
                 dff,  
                 pre_train,
                 input_vocab_size,  
                 maximun_position_encoding,
                 rate=0.1):
        super(Encoder, self).__init__()

        self.num_layers = num_layers
        self.d_model = d_model

        self.embedding = nn.Embedding(input_vocab_size, d_model)
        # self.embedding.weight.data.copy_(torch.from_numpy(pre_train))
        self.pos_encoding = positional_encoding(maximun_position_encoding,
                                                d_model)  # =>[1, max_pos_encoding, d_model=512]

        # self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate).cuda() for _ in range(num_layers)] # 不行
        self.enc_layers = torch.nn.ModuleList([EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

        self.dropout = torch.nn.Dropout(rate)

    # x [b, inp_seq_len]
    # mask [b, 1, 1, inp_sel_len]
    def forward(self, x, mask):
        inp_seq_len = x.shape[-1]

        # adding embedding and position encoding
        x = self.embedding(x)  # [b, inp_seq_len]=>[b, inp_seq_len, d_model]
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        pos_encoding = self.pos_encoding[:, :inp_seq_len, :]
        pos_encoding = pos_encoding.cuda()  # ###############
        x += pos_encoding  # [b, inp_seq_len, d_model]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, mask)  # [b, inp_seq_len, d_model]=>[b, inp_seq_len, d_model]
        return x  # [b, inp_seq_len, d_model]


In [96]:
class Decoder(torch.nn.Module):
    def __init__(self,
                 num_layers,  
                 d_model,
                 num_heads,
                 dff,  
                 target_vocab_size,  
                 maximun_position_encoding,
                 rate=0.1):
        super(Decoder, self).__init__()

        self.num_layers = num_layers
        self.d_model = d_model

        self.embedding = torch.nn.Embedding(num_embeddings=target_vocab_size, embedding_dim=d_model)
        self.pos_encoding = positional_encoding(maximun_position_encoding,
                                                d_model)  # =>[1, max_pos_encoding, d_model=512]

        self.dec_layers = torch.nn.ModuleList([DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

        self.dropout = torch.nn.Dropout(rate)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask,bertout,b_m):
        targ_seq_len = x.shape[-1]
        attention_weights = {}
        x = self.embedding(x)  # [b, targ_seq_len]=>[b, targ_seq_len, d_model]
        x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        pos_encoding = self.pos_encoding[:, :targ_seq_len, :]  # [b, targ_seq_len, d_model]
        pos_encoding = pos_encoding.cuda() # ###############
        x += pos_encoding  # [b, inp_seq_len, d_model]
        x = self.dropout(x)
        for i in range(self.num_layers):
            # compare to vanilla decoder, additionally feed the BERT padding mask and BERT output
            x, attn_block1, attn_block2 = self.dec_layers[i](x, enc_output, look_ahead_mask, padding_mask,bertout,b_m)
            attention_weights[f'decoder_layer{i + 1}_block1'] = attn_block1
            attention_weights[f'decoder_layer{i + 1}_block2'] = attn_block2

        return x, attention_weights

In [97]:
class Transformer(torch.nn.Module):
    def __init__(self,
                 num_layers,  
                 d_model,
                 num_heads,
                 dff,  
                 input_vocab_size, 
                 target_vocab_size,  
                 pe_input,  # input max_pos_encoding
                 pe_target,
                 # input max_pos_encoding
                 rate=0.1):
        super(Transformer, self).__init__()
        # initialize BERT model
        self.bert = BertModel.from_pretrained('bert-base-cased').to(device)
        self.encoder = Encoder(num_layers,
                               d_model,
                               num_heads,
                               dff,
                               input_vocab_size,
                               pe_input,
                               rate)
        self.decoder = Decoder(num_layers,
                               d_model,
                               num_heads,
                               dff,
                               target_vocab_size,
                               pe_target,
                               rate)
        self.final_layer = torch.nn.Linear(d_model, target_vocab_size)

    def forward(self, bertinp, inp, targ, enc_padding_mask, look_ahead_mask, dec_padding_mask,b_m):
        # get output of last hidden layer of BERT, and we don't fine-tune BERT in this project
        with torch.no_grad():
          bertout = self.bert(bertinp)[0][-1]
        enc_output = self.encoder(inp, enc_padding_mask)  # =>[b, inp_seq_len, d_model]
        # additionally feed the BERT padding mask and BERT output
        dec_output, attention_weights = self.decoder(targ, enc_output, look_ahead_mask, dec_padding_mask,bertout,b_m)
        final_output = self.final_layer(dec_output)  # =>[b, targ_seq_len, target_vocab_size]
        return final_output, attention_weights

In [98]:
loss_object = torch.nn.CrossEntropyLoss(reduction='none')
# real [batch_size, targ_seq_len]
# pred [batch_size, targ_seq_len, target_vocab_size]
def mask_loss_func(real, pred,pad):
    # print(real.shape, pred.shape)
    # _loss = loss_object(pred, real) # [batch_size, targ_seq_len]
    _loss = loss_object(pred.transpose(-1, -2), real)  # [batch_size, targ_seq_len]
    mask = torch.logical_not(real.eq(pad)).type(_loss.dtype)  # [batch_size, targ_seq_len] 
    _loss *= mask
    return _loss.sum() / mask.sum().item()
def mask_accuracy_func(real, pred,pad):
    _pred = pred.argmax(dim=-1)  # [batch_size, targ_seq_len, target_vocab_size]=>[b, targ_seq_len]
    corrects = _pred.eq(real)  # [batch_size, targ_seq_len] bool value
    mask = torch.logical_not(real.eq(pad))  # [batch_size, targ_seq_len] where index != 1 
    corrects *= mask
    return corrects.sum().float() / mask.sum().item()

In [99]:
# def create_mask(bret_inp, inp, targ):
#     # encoder padding mask
#     enc_padding_mask = create_padding_mask(inp,1)  # =>[b,1,1,inp_seq_len] mask=1的位置为pad

#     # decoder's first attention block(self-attention)
#     # 使用的padding create_mask & look-ahead create_mask
#     look_ahead_mask = create_look_ahead_mask(targ.shape[-1])  # =>[targ_seq_len,targ_seq_len] ##################
#     dec_targ_padding_mask = create_padding_mask(targ,1)  # =>[b,1,1,targ_seq_len]
#     combined_mask = torch.max(look_ahead_mask, dec_targ_padding_mask)  # 结合了2种mask =>[b,1,targ_seq_len,targ_seq_len]

#     # decoder's second attention block(encoder-decoder attention) 使用的padding create_mask
#     # 【注意】：这里的mask是用于遮挡encoder output的填充pad，而encoder的输出与其输入shape都是[b,inp_seq_len,d_model]
#     # 所以这里mask的长度是inp_seq_len而不是targ_mask_len
#     dec_padding_mask = create_padding_mask(inp,1)  # =>[b,1,1,inp_seq_len] mask=1的位置为pad

#     return enc_padding_mask, combined_mask, dec_padding_mask ,create_padding_mask(inp,0)


In [100]:
def validate_step(model, bretinp,inp, targ):
    targ_inp = targ[:, :-1]
    targ_real = targ[:, 1:]
    tem = inp.shape[1]
    bretinp = bretinp[:,:tem]
    enc_padding_mask, combined_mask, dec_padding_mask,b_mask = create_mask(bretinp,inp, targ_inp)
    bretinp = bretinp.to(device)
    inp = inp.to(device)
    b_mask = b_mask.to(device)

    targ_inp = targ_inp.to(device)
    targ_real = targ_real.to(device)
    enc_padding_mask = enc_padding_mask.to(device)
    combined_mask = combined_mask.to(device)
    dec_padding_mask = dec_padding_mask.to(device)

    model.eval()  

    with torch.no_grad():
        # forward
        prediction, _ = model(bretinp,inp, targ_inp, enc_padding_mask, combined_mask, dec_padding_mask,b_mask)
    
        
        val_loss = mask_loss_func(targ_real, prediction,1)
        val_metric = mask_accuracy_func(targ_real, prediction,1)
        
    return val_loss.item(), val_metric.item() 


In [101]:
def train_step(model,bretinp, inp, targ):
    tem = inp.shape[1]
    bretinp = bretinp[:,:tem]
    targ_inp = targ[:, :-1]
    targ_real = targ[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask,b_mask = create_mask(bretinp,inp, targ_inp)
    b_mask = b_mask.to(device)
    bretinp = bretinp.to(device)
    inp = inp.to(device)
    targ_inp = targ_inp.to(device)
    targ_real = targ_real.to(device)
    enc_padding_mask = enc_padding_mask.to(device)
    combined_mask = combined_mask.to(device)
    dec_padding_mask = dec_padding_mask.to(device)
    model.train()  # 设置train mode
    optimizer.zero_grad()  # 梯度清零

    # forward
    prediction, _ = model(bretinp,inp, targ_inp, enc_padding_mask, combined_mask, dec_padding_mask,b_mask)
    loss = mask_loss_func(targ_real, prediction,1)
    metric = mask_accuracy_func(targ_real, prediction,1)

    # backward
    loss.backward()  
    optimizer.step()  

    return loss.item(), metric.item()


In [102]:
EPOCHS = 20 # 50 # 30  # 20

print_trainstep_every = 50  # 每50个step做一次打印

metric_name = 'acc'
# df_history = pd.DataFrame(columns=['epoch', 'loss', metric_name]) # 记录训练历史信息
# df_history = pd.DataFrame(columns=['epoch', 'loss', metric_name, 'val_loss', 'val_' + metric_name,'test_loss', 'test_' + metric_name, 'bleu' ])


def printbar():
    nowtime = datetime.datetime.now().strftime('%Y-%m_%d %H:%M:%S')
    print('\n' + "=========="*8 + '%s'%nowtime)


In [103]:

import timeit


def train_model(model, epochs,datat,datav,datatest,datatest1,datatest2 ,print_every,BATCH_SIZE):
    starttime = time.time()
    print('*' * 27, 'start training...')
    printbar()

    best_acc = 0.
    df_history = pd.DataFrame(columns=['epoch', 'loss', metric_name, 'val_loss', 'val_' + metric_name,'test_loss', 'test_' + metric_name, 'test1_loss', 'test1_' + metric_name,'test2_loss', 'test2_' + metric_name])
    for epoch in range(1,epochs+1):
        # load data
        val_dataloader =  torch.utils.data.DataLoader(datav, batch_size = 256 ,shuffle = False,collate_fn= collate_fn)
        test_dataloader =  torch.utils.data.DataLoader(datatest, batch_size = 256 ,shuffle = False,collate_fn= collate_fn)
        test_dataloader1 =  torch.utils.data.DataLoader(datatest1, batch_size = 256 ,shuffle = False,collate_fn= collate_fn)
        test_dataloader2 =  torch.utils.data.DataLoader(datatest2, batch_size = 256 ,shuffle = False,collate_fn= collate_fn)
        train_dataloader = torch.utils.data.DataLoader(datat, batch_size = BATCH_SIZE ,shuffle = True,collate_fn= collate_fn)
        loss_sum = 0.
        metric_sum = 0.
        start = timeit.default_timer()
        for step, (bretinp,inp, targ) in enumerate(train_dataloader,start = 1):

            loss, metric = train_step(model,bretinp, inp, targ)

            loss_sum += loss
            metric_sum += metric

            if step % print_every == 0:
                print('*' * 8, f'[step = {step}] loss: {loss_sum / step:.3f}, {metric_name}: {metric_sum / step:.3f}')
            
        stop = timeit.default_timer()

        print('Time: ', stop - start) 
        
        scheduler.step()
        val_loss_sum = 0.
        val_metric_sum = 0.
        for val_step, (bretinp,inp, targ) in enumerate(val_dataloader, start=1):
            loss, metric = validate_step(model,bretinp, inp, targ)

            val_loss_sum += loss
            val_metric_sum += metric

        test_loss_sum = 0.
        test_metric_sum = 0.
        bleu = []
        for test_step, (bretinp,inp, targ) in enumerate(test_dataloader, start=1):
            loss, metric = validate_step(model,bretinp, inp, targ)

            test_loss_sum += loss
            test_metric_sum += metric

        test_loss_sum1 = 0.
        test_metric_sum1 = 0.
        bleu = []
        for test_step1, (bretinp,inp, targ) in enumerate(test_dataloader1, start=1):
            # inp [64, 10] , targ [64, 10]
            loss, metric = validate_step(model,bretinp, inp, targ)

            test_loss_sum1 += loss
            test_metric_sum1 += metric
        test_loss_sum2 = 0.
        test_metric_sum2 = 0.
        bleu = []
        for test_step2, (bretinp,inp, targ) in enumerate(test_dataloader2, start=1):
            # inp [64, 10] , targ [64, 10]
            loss, metric = validate_step(model,bretinp, inp, targ)

            test_loss_sum2 += loss
            test_metric_sum2 += metric
        if epoch<1000:
          record = (epoch, loss_sum/step, metric_sum/step, val_loss_sum/val_step, val_metric_sum/val_step,
                  test_loss_sum/test_step, test_metric_sum/test_step,test_loss_sum1/test_step1, test_metric_sum1/test_step1,test_loss_sum2/test_step2, test_metric_sum2/test_step2)
          df_history.loc[epoch - 1] = record
          print('EPOCH = {} loss: {:.3f}, {}: {:.3f}, val_loss: {:.3f}, val_{}: {:.3f},test_loss: {:.3f}, test_{}: {:.3f}, test1_loss: {:.3f}, test1_{}: {:.3f},test2_loss: {:.3f}, test2_{}: {:.3f}'.format(
            record[0], record[1], metric_name, record[2], record[3], metric_name, record[4],record[5],metric_name, record[6],record[7],metric_name,record[8],record[9],metric_name,record[10]))
          printbar()
          if len(df_history)>=11:
            if record[3]>=df_history.loc[epoch - 11][3]:
              break
          else:
            continue
        
    print('finishing training...')
    endtime = time.time()
    time_elapsed = endtime - starttime
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return df_history



In [104]:


def tokenzier_decode1(sentence_ids, vocab):
    sentence = [vocab.itos[id] for id in sentence_ids if id<len(vocab)]
    # print(sentence)
    return " ".join(sentence[1:])

def evaluate(model, inp_sentence):
    model.eval()  # 设置eval mode
    # print(tokenizer(inp_sentence))
    inp = [SRC_TEXT.vocab.stoi['[CLS]']] + [SRC_TEXT.vocab.stoi[i] for i in tz.tokenize(inp_sentence)] +[SRC_TEXT.vocab.stoi['[SEP]']]
    bretinp =  torch.tensor([tz.convert_tokens_to_ids(['[CLS]'] + tz.tokenize(inp_sentence) + ['[SEP]'])]) # 转化为索引
    # tem = len(bretinp[0])
    # inp = [SRC_TEXT.vocab.stoi['[CLS]']]+[SRC_TEXT.vocab.stoi[i] for i in inp_sentence.split()]+[SRC_TEXT.vocab.stoi['[SEP]']]
    # inp += (tem-len(inp))*[0]
    
    inp = torch.unsqueeze(torch.tensor(inp),0)
    
    decoder_input = [TARG_TEXT.vocab.stoi['[CLS]']]
    decoder_input = torch.tensor(decoder_input).unsqueeze(0)  # =>[b=1,seq_len=1]
    # print(decoder_input.shape)
    m = nn.LogSoftmax(dim=-1)

    with torch.no_grad():
        enc_padding_mask, combined_mask, dec_padding_mask,b_mask = create_mask(bretinp.cpu(),inp.cpu(), decoder_input.cpu())
        bretinp = bretinp.to(device)  
        inp = inp.to(device)
        enc_padding_mask = enc_padding_mask.to(device)
        enout = model.encoder(inp, enc_padding_mask) 
        bertout = model.bert(bretinp)[0][-1]
        for i in range(MAX_LENGTH + 2):
            # enc_padding_mask, combined_mask, dec_padding_mask,b_mask = create_mask(inp_sentence_ids.cpu(), decoder_input.cpu()) ################
            # [b,1,1,inp_seq_len], [b,1,targ_seq_len,inp_seq_len], [b,1,1,inp_seq_len]
            enc_padding_mask, combined_mask, dec_padding_mask,b_mask = create_mask(bretinp.cpu(),inp.cpu(), decoder_input.cpu())
            b_mask = b_mask.to(device)
            
            decoder_input = decoder_input.to(device)
            
            combined_mask = combined_mask.to(device)
            dec_padding_mask = dec_padding_mask.to(device)

          # bretinp,inp, targ_inp, enc_padding_mask, combined_mask, dec_padding_mask,b_mask
            predictions, attention_weights = model.decoder(decoder_input, enout, combined_mask, dec_padding_mask,bertout,b_mask)
           
            predictions = model.final_layer(predictions)
            prediction = m(predictions[:, -1:, :])  # =>[b=1, 1, target_vocab_size]
            
            prediction_id = torch.argmax(prediction, dim=-1)  # => [b=1, 1]
            # print('prediction_id:', prediction_id, prediction_id.dtype) # torch.int64
            if prediction_id.squeeze().item() == TARG_TEXT.vocab.stoi['[SEP]']:
                return decoder_input.squeeze(dim=0), attention_weights

            decoder_input = torch.cat([decoder_input, prediction_id],
                                      dim=-1)  # [b=1,targ_seq_len=1]=>[b=1,targ_seq_len=2]


    return decoder_input.squeeze(dim=0), attention_weights

In [ ]:
# datat = list(zip(enco,final_trains))
datat = list(zip(enco,final_trains_i,final_trains))
# datav = list(zip(encoval,final_val_s))
datav = list(zip(encoval,final_val_i,final_val_s))
# datatest = list(zip(encotest,final_test_s))
datatest = list(zip(encotest,final_test_i,final_test_s))
datatest1 = list(zip(encotest1,final_test_i1,final_test_s))
datatest2 = list(zip(encotest2,final_test_i2,final_test_s))
num_layers = 4
d_model = 768
dff =  800
num_heads = 6



input_vocab_size = len(SRC_TEXT.vocab)
target_vocab_size = len(TARG_TEXT.vocab) # 2591
dropout_rate = 0.25
print(target_vocab_size)
torch.cuda.empty_cache()
dual_transformer = Transformer(num_layers,
                          d_model,
                          num_heads,
                          dff,
                           input_vocab_size,
                          target_vocab_size,
             input_vocab_size,
                          pe_target=target_vocab_size,
                          
                          rate=dropout_rate)
dual_transformer = dual_transformer.to(device)



optimizer = torch.optim.Adam(dual_transformer.parameters(),lr=0.0003)
scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer , 3,
                gamma =0.8)
df_history = train_model(dual_transformer, 20, datat,datav,datatest,datatest1,datatest2,100,256)

print(df_history)

19063
*************************** start training...

================================================================================2021-09_26 19:42:04
******** [step = 100] loss: 2.392, acc: 0.653
******** [step = 200] loss: 1.962, acc: 0.732
Time:  140.17464453799994
EPOCH = 1 loss: 1.954, acc: 0.733, val_loss: 1.541, val_acc: 0.815,test_loss: 1.551, test_acc: 0.814, test1_loss: 1.627, test1_acc: 0.802,test2_loss: 1.708, test2_acc: 0.789

================================================================================2021-09_26 19:45:35
******** [step = 100] loss: 1.301, acc: 0.839
******** [step = 200] loss: 1.231, acc: 0.849
Time:  139.957120756
EPOCH = 2 loss: 1.228, acc: 0.849, val_loss: 1.158, val_acc: 0.861,test_loss: 1.167, test_acc: 0.860, test1_loss: 1.301, test1_acc: 0.844,test2_loss: 1.435, test2_acc: 0.827

================================================================================2021-09_26 19:49:06
******** [step = 100] loss: 1.185, acc: 0.826
******** [step = 200

In [ ]:
all_re = []
gt= [[i[1].split()]for i in test_pairs]
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
total = 0
for i in tqdm(range(len(test_pairs))):
       
      
        references = gt[i]
        pred_result, _ = evaluate(transformer1, test_pairs[i][0])
        candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split()

        sbs = sentence_bleu(references, candidates,smoothing_function=smoothie)
        all_re.append(sbs)
        total+= sbs
total*100/len(test_pairs)

In [ ]:

gt= [[i[1].split()]for i in test_pairs1]
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
total = 0
for i in tqdm(range(len(test_pairs1))):
       
      
        references = gt[i]
        pred_result, _ = evaluate(transformer1, test_pairs1[i][0])
        candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split()


        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(test_pairs)

In [ ]:

gt= [[i[1].split()]for i in test_pairs2]
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
total = 0
for i in tqdm(range(len(test_pairs2))):
       
      
        references = gt[i]
        pred_result, _ = evaluate(transformer1, test_pairs2[i][0])
        candidates = tokenzier_decode1(pred_result, TARG_TEXT.vocab).split()


        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(test_pairs)

In [ ]:
def batch_translate(pairs,n):
    for i in range(n):
        pair = np.random.choices(pairs)
        print('input:', pair[0])
        print('target:', pair[1])
        pred_result, _ = evaluate(transformer1, pair[0])
        pred_sentence = tokenzier_decode1(pred_result, TARG_TEXT.vocab)
        print('pred:', pred_sentence)
        print('')


In [ ]:
batch_translate(test_pairs,10)